In [ ]:
import cashladder

In [ ]:
import lusid
import lusid.models as models
import os
from msrest.authentication import BasicTokenAuthentication

api_url = os.getenv("FBN_LUSID_API_URL")
api_token = {"access_token": os.environ["ACCESS_TOKEN"]}
credentials = BasicTokenAuthentication(api_token)
client = lusid.LUSIDAPI(credentials, api_url)

import uuid
from datetime import datetime
from datetime import timedelta
import pytz
from collections import namedtuple

def assert_response_is_not_error(expected_type, response):

    if isinstance(response, models.ErrorResponse):
        self.fail(response.detailed_message)
    elif isinstance(response, expected_type):
        return response
    else:
        self.fail("unknown response: {0}".format(type(response)))
            
def build_trade(trade_spec):
    return models.TradeDto(
        trade_id=str(uuid.uuid4()),
        type="Buy",
        security_uid=trade_spec.id,
        settlement_currency=trade_spec.currency,
        trade_date=trade_spec.trade_date,
        settlement_date=trade_spec.trade_date + timedelta(days=2),
        units=100,
        trade_price=trade_spec.price,
        total_consideration=100 * trade_spec.price,
        source="Client")

TradeSpec = namedtuple('TradeSpec', 'id price trade_date settlement_date currency')

def create_portfolio_with_trades(scope, name, date, trades):
    scope = "finbourne"
    guid = str(uuid.uuid4())
    portfolio_code = "id-{0}".format(guid)
    effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)

    request = models.CreatePortfolioRequest("portfolio-{0}".format(guid), portfolio_code, "GBP", effective_date)

    #   create the portfolio
    result = client.create_portfolio(scope, request)
    portfolio_id = result.id.code

    assert_response_is_not_error(models.PortfolioDto, result)

    #   build and add trades
    new_trades = list(map(build_trade, trades))
    add_trades_result = client.upsert_trades(scope, portfolio_id, new_trades)
    assert_response_is_not_error(models.UpsertPortfolioTradesDto, add_trades_result)
    
    return portfolio_code

In [ ]:
from datetime import datetime

effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)
trades = [
    TradeSpec("FIGI_BBG001S7Z574", 101, datetime(2018, 1, 1, tzinfo=pytz.utc), datetime(2018, 1, 4, tzinfo=pytz.utc), "GBP"),       
    TradeSpec("FIGI_BBG001S7Z574", 102, datetime(2018, 1, 2, tzinfo=pytz.utc), datetime(2018, 1, 5, tzinfo=pytz.utc), "GBP"),
    TradeSpec("FIGI_BBG001S7Z574", 103, datetime(2018, 1, 3, tzinfo=pytz.utc), datetime(2018, 1, 6, tzinfo=pytz.utc), "GBP"),
    TradeSpec("FIGI_BBG001S7Z574", 104, datetime(2018, 1, 1, tzinfo=pytz.utc), datetime(2018, 1, 4, tzinfo=pytz.utc), "USD"),
    TradeSpec("FIGI_BBG001SRKHW2", 105, datetime(2018, 1, 2, tzinfo=pytz.utc), datetime(2018, 1, 5, tzinfo=pytz.utc), "USD"),
    TradeSpec("FIGI_BBG001SRKHW2", 106, datetime(2018, 1, 3, tzinfo=pytz.utc), datetime(2018, 1, 6, tzinfo=pytz.utc), "USD"),
    TradeSpec("FIGI_BBG001S7Z574", 107, datetime(2018, 1, 1, tzinfo=pytz.utc), datetime(2018, 1, 4, tzinfo=pytz.utc), "EUR"),
    TradeSpec("FIGI_BBG000005547", 108, datetime(2018, 1, 2, tzinfo=pytz.utc), datetime(2018, 1, 5, tzinfo=pytz.utc), "EUR"),      
    TradeSpec("FIGI_BBG001SRKHW2", 109, datetime(2018, 1, 3, tzinfo=pytz.utc), datetime(2018, 1, 6, tzinfo=pytz.utc), "EUR"),
    
]

created_code = create_portfolio_with_trades(scope="finbourne", name="demo", date=effective_date, trades=trades)

In [ ]:
import cashladder as c 
cl_date = '2018-01-04'
c.cash_ladder(date=cl_date, scope='finbourne',portfolio=created_code)

In [ ]:
acl = c.alt_cash_ladder(date=cl_date,scope='finbourne',portfolio=created_code)
chart_data = acl[['Currency','Cash Date','balance']].drop_duplicates(['Currency','Cash Date'],keep='last')
chart_data = chart_data.pivot(index="Currency", columns="Cash Date", values="balance").fillna(0)

In [ ]:
chart_data

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

ind = np.arange(chart_data.columns.size)
width = 0.1

fig, ax = plt.subplots()

for ccy, x in zip(chart_data.index, [-.2, 0, .2]):
    ax.bar(ind + x/2, chart_data.loc[ccy,:].asobject, width, label=ccy)
    
ax.set_xticks(ind)
ax.set_xticklabels(chart_data.columns.values)
ax.legend()

plt.show()